In [3]:
# Installs geemap package
import subprocess

try:
        import geemap
except ImportError:
        print('geemap package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Authenticates and initializes Earth Engine
import ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

# read ROIs from geojson file

In [4]:
import json

roiFile=r".\files\Multi_Class_Land_Cover_Change_AOIs.geojson"

with open(roiFile) as f:
    data = json.load(f)
    
for feature in data['features']:
    #print(feature['properties'])
    #featureGEE=ee.Feature(geojsonDict)
    
    '''convert to gee feature'''
    #jsonDict = eval(geom.to_json())
    #geojsonDict = jsonDict['features'][0]

    cor=feature['geometry']['coordinates']
         
    aoi = ee.Geometry.Polygon([cor[0][0][0:2], cor[0][1][0:2], cor[0][2][0:2], cor[0][3][0:2],cor[0][4][0:2]], None, False)
    #print(aoi)

ee.Geometry({
  "type": "Invocation",
  "arguments": {
    "coordinates": [
      [
        -55.546875,
        -11.824341483849047
      ],
      [
        -55.546875,
        -11.78132529611228
      ],
      [
        -55.5029296875,
        -11.78132529611228
      ],
      [
        -55.5029296875,
        -11.824341483849047
      ],
      [
        -55.546875,
        -11.824341483849047
      ]
    ],
    "geodesic": false
  },
  "functionName": "GeometryConstructors.Polygon"
})
ee.Geometry({
  "type": "Invocation",
  "arguments": {
    "coordinates": [
      [
        14.2822265625,
        12.854648905588952
      ],
      [
        14.2822265625,
        12.897489183755896
      ],
      [
        14.326171875,
        12.897489183755896
      ],
      [
        14.326171875,
        12.854648905588952
      ],
      [
        14.2822265625,
        12.854648905588952
      ]
    ],
    "geodesic": false
  },
  "functionName": "GeometryConstructors.Polygon"
})
ee.Geometry({


In [5]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

# visualize one aoi

In [20]:
feature =data['features'][0]
cor=feature['geometry']['coordinates']     
aoi = ee.Geometry.Polygon([cor[0][0][0:2], cor[0][1][0:2], cor[0][2][0:2], cor[0][3][0:2],cor[0][4][0:2]], None, False)
    

In [21]:
# Display the polygons by adding them to the map.
Map.centerObject(aoi,10)
Map.addLayer(aoi, {'color': 'FF0000'})

# download monthly s2 data for one aoi 

In [63]:
timePeriod = ee.DateRange('2018-01-01', '2019-02-01')

s2_bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
#['Blue', 'Green', 'Red', 'Red Edge 1', 'Red Edge 2', 'Red Edge 3', 'NIR', 'Red Edge 4', 'SWIR1', 'SWIR2', 'QA60']

In [12]:
def get_utm_epsg(lon, lat):
	#coords = feature.geometry().coordinates()
	#lon = ee.Number(coords.get(0))
	#lat = ee.Number(coords.get(1))
	epsg = ee.Number(32700).subtract(lat.add(45).divide(90).round().multiply(100)).add(lon.add(183).divide(6).round()).uint16()
	#print(epsg.getInfo())
	return e.String("EPSG:").cat(ee.String(str(epsg.getInfo())))#ee.String("EPSG:").cat

In [57]:
#ee.ImageCollection("COPERNICUS/S2_SR")
#https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR
               
def get_s1_mosaic(city):
    #input is a gee feature
    
    roi = city.geometry()

    def clip_roi(img):
        return img.clip(roi).unmask()

    collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterDate(timePeriod).filterBounds(roi).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .select(s1_bands) \
    .map(clip_roi) \
    .median()
    return collection
def getS2(aoi, timePeriod):
	# Define function to mask out clouds and cloud shadows in images.
	# Use CFmask band included in USGS Landsat SR image product.
	def fmask(img):
		cloudOpaqueBitMask = 1 << 10
		cloudCirrusBitMask = 1 << 11
		qa = img.select('QA60')
		mask = qa.bitwiseAnd(cloudOpaqueBitMask).eq(0) \
            .And(qa.bitwiseAnd(cloudCirrusBitMask).eq(0))
		return(img.updateMask(mask))
               
	image = ee.ImageCollection('COPERNICUS/S2_SR') \
	.filterDate(timePeriod).select(s2_bands) \
	.filterBounds(aoi).map(fmask).median()

	return image.clip(aoi)


In [64]:
#s2 = getS2(aoi, timePeriod)
s2 = ee.ImageCollection('COPERNICUS/S2') \
	.filterDate(timePeriod).select(s2_bands) \
	.filterBounds(aoi).median().clip(aoi)

#print(s2.getInfo())

'for debug'
#print(s2.getInfo())
Map.addLayer(s2, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000}, "S2")

{'type': 'Image', 'bands': [{'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [1, 1], 'origin': [-56, -12], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [1, 1], 'origin': [-56, -12], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [1, 1], 'origin': [-56, -12], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [1, 1], 'origin': [-56, -12], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [1, 1], 'origin': [-56, -12], 'crs': 'EPSG:4326', 'crs_transform': [1,

In [ ]:
geemap.ee_export_image(s2, filename=filename, scale=10, region=aoi, crs=ee.Projection(feature_epsg.get("EPSG")), file_per_band=False)#, region=roi